In [2]:
import getpass
import pandas as pd
import numpy as np
import urllib.parse
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Markdown as md

from shapely import wkb
from scipy.spatial import distance_matrix
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, value
import cvxpy as cp
from shapely import wkt
from shapely.geometry import LineString, Point

pd.set_option('display.max_columns', None)

In [3]:
edm_address = getpass.getpass(prompt='EDM server address: ') 

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ') 
edm_password = getpass.getpass(prompt='Password: ') 
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purpose.
del edm_name, edm_password

EDM server address:  ········



EDM login information


Username:  ········
Password:  ········


In [4]:
def query_to_df(result):
    """
    Transform sql query result to dataframe, and
    expand out `meta` column into separate columns.
    """
    
    # Turn a query output into a Python dataframe.
    df = result.DataFrame()

    # Separate the `meta` column currently saved as JSONB into individual columns.
    # Replace NaN values to empty string for ease of spotting non-NaN values.
    df = pd.concat([df.drop(['meta'], axis=1),
                    df['meta'].apply(pd.Series).fillna('')], axis=1)
    
    # Transform the hex string form of geometry to coordinates.
    df['geometry'] = df['geometry'].apply(lambda x: wkb.loads(x, hex=True)) 
    
    return df

In [5]:
# User input for the grid.
grid_id = input('Enter grid ID: ') # awefice

Enter grid ID:  awefice


In [6]:
result = %sql SELECT grid_element_id, \
                        meta, \
                        is_producer, \
                        geometry, \
                        phases\
                FROM grid_element\
                WHERE grid_id = '{grid_id}'\
                    AND type = 'Transformer';

# Convert the results to a data frame.
df_transformers = result.DataFrame()

# Pull out the information from `meta` column saved as JSONB.
df_transformers = pd.concat([df_transformers.drop(['meta'], axis=1),
                             df_transformers['meta'].apply(pd.Series)], axis=1)

# Choose the relevant columns to display transformers. 
df_transformers = df_transformers[['grid_element_id', 'ownership', 'rating_kva', 
                                   'phases', 'voltage_level', 'commission_date', 
                                   'primary_voltage', 'secondary_voltage','is_producer','geometry']]

# Display the results.
df_transformers

Transformers_info = df_transformers[['grid_element_id','is_producer','rating_kva', 'geometry']].set_index('grid_element_id')
Transformers_info['rating_kva'] = Transformers_info['rating_kva']
Transformers_info
Transformers_info

,is_producer,rating_kva,geometry
grid_element_id,,,
transformer_16,False,5.0,0101000020E6100000FBE59315C3C65EC00CF5264C39A2...
transformer_2,True,20000.0,0101000020E61000002B8A5759DBC65EC07AA4038A3FA2...
transformer_26,False,10.0,0101000020E610000083DC4598A2C65EC0EF130F4138A2...
transformer_31,False,30.0,0101000020E61000002B508BC1C3C65EC09C3C3CB62BA2...
transformer_36,False,50.0,0101000020E61000004D81CCCEA2C65EC0BA9955C82AA2...
transformer_43,False,75.0,0101000020E61000002463B5F97FC65EC0843DD45337A2...
transformer_47,False,150.0,0101000020E61000001F2E39EE94C65EC0A6A45E2224A2...
transformer_6,True,20000.0,0101000020E61000005A0EF450DBC65EC07EB981C843A2...
transformer_63,False,300.0,0101000020E61000009885764EB3C65EC04A8D3B6B58A2...


In [7]:
Ts_Producer = Transformers_info[Transformers_info['is_producer'] == True].index.tolist()
Ts_Producer

['transformer_2', 'transformer_6']

In [8]:
Ts_Consumer = Transformers_info[Transformers_info['is_producer'] == False].index.tolist()
Ts_Consumer

['transformer_16',
 'transformer_26',
 'transformer_31',
 'transformer_36',
 'transformer_43',
 'transformer_47',
 'transformer_63',
 'transformer_92']

In [9]:
Transformers_info_Consumer = Transformers_info[Transformers_info['is_producer']==False]
Transformers_info_Consumer

,is_producer,rating_kva,geometry
grid_element_id,,,
transformer_16,False,5.0,0101000020E6100000FBE59315C3C65EC00CF5264C39A2...
transformer_26,False,10.0,0101000020E610000083DC4598A2C65EC0EF130F4138A2...
transformer_31,False,30.0,0101000020E61000002B508BC1C3C65EC09C3C3CB62BA2...
transformer_36,False,50.0,0101000020E61000004D81CCCEA2C65EC0BA9955C82AA2...
transformer_43,False,75.0,0101000020E61000002463B5F97FC65EC0843DD45337A2...
transformer_47,False,150.0,0101000020E61000001F2E39EE94C65EC0A6A45E2224A2...
transformer_63,False,300.0,0101000020E61000009885764EB3C65EC04A8D3B6B58A2...
transformer_92,False,750.0,0101000020E61000005B94D92093C65EC0F1C5CD5F57A2...


In [10]:
DF_Distancias = pd.DataFrame(0, index=Ts_Consumer, columns=Ts_Producer)
DF_Distancias

,transformer_2,transformer_6
transformer_16,0,0
transformer_26,0,0
transformer_31,0,0
transformer_36,0,0
transformer_43,0,0
transformer_47,0,0
transformer_63,0,0
transformer_92,0,0


In [11]:
for producer in Ts_Producer:
    grid_element_id = producer
    trace_option = 'all sources'
    result = %sql SELECT * \
            FROM grid_get_sources('{grid_id}', '{grid_element_id}', 'false')

    #Lines
    df = query_to_df(result)
    
    df_filtrado = df[
        (df['type'] == 'ACLineSegment') &
        (df['phases'] == 'ABC') & (df['voltage_level']=='MV')
    ][['grid_element_id', 'geometry']]
    
    for consumer in Ts_Consumer:
        punto = wkb.loads(bytes.fromhex(Transformers_info_Consumer.loc[consumer, 'geometry']))
        Distancias=pd.DataFrame()
        Distancias['distancia'] = df_filtrado['geometry'].apply(lambda line: punto.distance(line))
        minima=Distancias['distancia'].min()
        DF_Distancias.at[consumer, producer] = minima
DF_Distancias

C:\Users\puerc\AppData\Local\Temp\ipykernel_83304\1640385387.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.4898719453835265e-06' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  DF_Distancias.at[consumer, producer] = minima
C:\Users\puerc\AppData\Local\Temp\ipykernel_83304\1640385387.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0010692648457825745' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  DF_Distancias.at[consumer, producer] = minima


,transformer_2,transformer_6
transformer_16,0.000004,0.001069
transformer_26,0.000004,0.001342
transformer_31,0.000004,0.001222
transformer_36,0.000005,0.001668
transformer_43,0.000003,0.001528
transformer_47,0.000002,0.001573
transformer_63,0.000970,0.000006
transformer_92,0.000967,0.000006


In [12]:
Distances_array = DF_Distancias.values
Distances_array

array([[4.48987195e-06, 1.06926485e-03],
       [4.49683585e-06, 1.34249034e-03],
       [4.49770634e-06, 1.22158205e-03],
       [4.51672100e-06, 1.66778543e-03],
       [3.20526379e-06, 1.52838233e-03],
       [2.31335839e-06, 1.57314108e-03],
       [9.69704305e-04, 5.62513663e-06],
       [9.66662579e-04, 5.68465438e-06]])

In [14]:
Exp_Load_Producer=pd.DataFrame(columns=["transformer_id", "Load"])
for index in Ts_Producer:
    grid_element_id = index
    result = %sql SELECT ge.grid_element_id as transformer_id, \
                            tdss_c.timestamp at time zone 'America/Vancouver' as timestamp, \
                            SUM(tdss_c.value - COALESCE(tdss_p.value, 0)) as "total_kW", ge.meta \
                    FROM grid_element ge \
                    JOIN grid_get_downstream('{grid_id}', ge.grid_element_id, 'false') ggd\
                        ON ggd.grid_id = ge.grid_id \
                    JOIN grid_element_data_source geds_c \
                        ON geds_c.grid_element_id = ggd.grid_element_id \
                        AND geds_c.type = 'CONSUMER' \
                    JOIN ts_data_source_select(geds_c.grid_element_data_source_id, 'kWh') tdss_c \
                        ON true \
                    LEFT JOIN grid_element_data_source geds_p \
                        ON geds_p.grid_element_id = geds_c.grid_element_id \
                        AND geds_p.type = 'PRODUCER' \
                    LEFT JOIN ts_data_source_select(geds_p.grid_element_data_source_id, 'kWh') tdss_p \
                        ON tdss_p.timestamp = tdss_c.timestamp \
                    WHERE ge.grid_element_id = '{grid_element_id}' \
                        AND ggd.type = 'Meter' \
                    GROUP BY ge.grid_element_id, tdss_c.timestamp, ge.meta \
                    ORDER by 2;
    
    # Convert the results to a data frame.
    df_transformer_load = result.DataFrame()
    
    # Pull out the information from the `meta` column saved as JSONB.
    df_transformer_load = pd.concat([df_transformer_load.drop(['meta'], axis=1),
                                    df_transformer_load['meta'].apply(pd.Series)], axis=1)
    
    # Choose the relevant columns to display. 
    df_transformer_load = df_transformer_load[['transformer_id', 'total_kw']]
    
    # Expected load + 1.28 times sd (80% of cases) added to df
    
    new_input = pd.DataFrame([{
        "transformer_id": index,
        "Load": df_transformer_load['total_kw'].mean() + 1.28 * np.std(df_transformer_load['total_kw'])
    }])
    
    Exp_Load_Producer = pd.concat([Exp_Load_Producer, new_input], ignore_index=True)

Exp_Load_Producer

C:\Users\puerc\AppData\Local\Temp\ipykernel_81452\1887534556.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Exp_Load_Producer = pd.concat([Exp_Load_Producer, new_input], ignore_index=True)


,transformer_id,Load
0,transformer_2,75.757616
1,transformer_6,43.201233


In [13]:
Exp_Load_Consumer=pd.DataFrame(columns=["transformer_id", "Load"])
for index in Ts_Consumer:
    grid_element_id = index
    result = %sql SELECT ge.grid_element_id as transformer_id, \
                            tdss_c.timestamp at time zone 'America/Vancouver' as timestamp, \
                            SUM(tdss_c.value - COALESCE(tdss_p.value, 0)) as "total_kW", ge.meta \
                    FROM grid_element ge \
                    JOIN grid_get_downstream('{grid_id}', ge.grid_element_id, 'false') ggd\
                        ON ggd.grid_id = ge.grid_id \
                    JOIN grid_element_data_source geds_c \
                        ON geds_c.grid_element_id = ggd.grid_element_id \
                        AND geds_c.type = 'CONSUMER' \
                    JOIN ts_data_source_select(geds_c.grid_element_data_source_id, 'kWh') tdss_c \
                        ON true \
                    LEFT JOIN grid_element_data_source geds_p \
                        ON geds_p.grid_element_id = geds_c.grid_element_id \
                        AND geds_p.type = 'PRODUCER' \
                    LEFT JOIN ts_data_source_select(geds_p.grid_element_data_source_id, 'kWh') tdss_p \
                        ON tdss_p.timestamp = tdss_c.timestamp \
                    WHERE ge.grid_element_id = '{grid_element_id}' \
                        AND ggd.type = 'Meter' \
                    GROUP BY ge.grid_element_id, tdss_c.timestamp, ge.meta \
                    ORDER by 2;
    
    # Convert the results to a data frame.
    df_transformer_load = result.DataFrame()
    
    # Pull out the information from the `meta` column saved as JSONB.
    df_transformer_load = pd.concat([df_transformer_load.drop(['meta'], axis=1),
                                    df_transformer_load['meta'].apply(pd.Series)], axis=1)
    
    # Choose the relevant columns to display. 
    df_transformer_load = df_transformer_load[['transformer_id', 'total_kw']]
    
    # Expected load + 1.28 times sd (80% of cases) added to df
    
    new_input = pd.DataFrame([{
        "transformer_id": index,
        "Load": df_transformer_load['total_kw'].mean() + 1.28 * np.std(df_transformer_load['total_kw'])
    }])
    
    Exp_Load_Consumer = pd.concat([Exp_Load_Consumer, new_input], ignore_index=True)

Exp_Load_Consumer

C:\Users\puerc\AppData\Local\Temp\ipykernel_83304\2406804298.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Exp_Load_Consumer = pd.concat([Exp_Load_Consumer, new_input], ignore_index=True)


,transformer_id,Load
0,transformer_16,7.540082
1,transformer_26,6.854288
2,transformer_31,7.871367
3,transformer_36,14.515007
4,transformer_43,19.197438
5,transformer_47,33.197110
6,transformer_63,13.216529
7,transformer_92,31.795568


In [14]:
capacity=Transformers_info.loc[Transformers_info['is_producer'] == True, ['rating_kva']].to_numpy().ravel()

In [38]:
n_consumers = len(Ts_Consumer)
n_generators = len(Ts_Producer)


demands = Exp_Load_Consumer['Load'].to_numpy()
capacities = capacity

#Standarization demand

demands_standarized = demands/np.max(demands)
capacities_standardized = capacities/np.max(demands)

#Distances

distances = Distances_array

#std distnances

distances_standarized=distances/np.max(distances)

x = cp.Variable((n_consumers, n_generators), boolean=True) # Boolean variables
load = cp.sum(cp.multiply(demands_standarized[:, None], x), axis=0) # Product of demand by boolean variable
load_avg = cp.sum(load) / n_generators  # Average load attributed to producing generators
load_variance = cp.sum_squares(load - load_avg) # Variance of load attributed to producing generators
total_distance = cp.sum(cp.multiply(distances_standarized, x))   # distance multiplied by boolean variable

alpha = .1 # weight of distance
threshold = 0.8 # % of overload

objective = cp.Minimize(alpha * total_distance + (1-alpha) * load_variance)

constraints = [cp.sum(x[i, :]) == 1 for i in range(n_consumers)]
constraints += [load[j] <= capacities_standardized[j]*threshold for j in range(n_generators)]

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS_BB)

print("Problem status:", problem.status)
print('---------------------------------------------------')

# Resultados
df_result = pd.DataFrame(x.value, index=Ts_Consumer, columns=Ts_Producer)

assignments = np.argwhere(df_result > 0.9)

# Return to no standarized values

load_real = np.zeros(n_generators)
distance_real = 0

for i, j in assignments:
    load_real[j] += demands[i]

for i, j in assignments:
    distance_real += distances[i, j]

#pring Results

for i, j in assignments :
    consumidor = df_result.index[i]
    generador = df_result.columns[j]
    print(f" {consumidor} assigned to {generador}")
print('---------------------------------------------------')

for j in range(n_generators):
    print(f"Optimal Loads for {df_result.columns[j]}: {load_real[j]:.2f}")
    
print('---------------------------------------------------')

print("Total Distance:", distance_real)

print('---------------------------------------------------')

print("Value of the objective function:", problem.value)

Problem status: optimal
---------------------------------------------------
 transformer_16 assigned to transformer_2
 transformer_26 assigned to transformer_2
 transformer_31 assigned to transformer_2
 transformer_36 assigned to transformer_2
 transformer_43 assigned to transformer_6
 transformer_47 assigned to transformer_2
 transformer_63 assigned to transformer_6
 transformer_92 assigned to transformer_6
---------------------------------------------------
Optimal Loads for transformer_2: 69.98
Optimal Loads for transformer_6: 64.21
---------------------------------------------------
Total Distance: 0.0015600066163285406
---------------------------------------------------
Value of the objective function: 0.10712419497088471
